# SQL Data munging example

In this exercise, we will experiment with data in a sqlite database using pandas data queries.

## Slide example

Start by running the command from the slides.  The following 2 cells setup the database.

In [ ]:
!rm new.db
!sqlite3 new.db "create table customer \
(cid numeric, \
cust_name char(20), \
address varchar(256), \
primary key (cid));"

!sqlite3 new.db "create table product \
(pid numeric, \
prod_name char(20), \
price numeric, \
primary key (pid));"

!sqlite3 new.db "create table order_n \
(oid numeric, \
pid numeric references product, \
cid numeric references customer, \
quantity numeric, \
Primary key (oid));"

In [ ]:
!sqlite3 new.db 'insert into customer values (1,"Joe Klein","USA");'
!sqlite3 new.db 'insert into customer values (2,"Rob Smith","CAN");'
!sqlite3 new.db 'insert into product values (1,"Pencil",1.23);'
!sqlite3 new.db 'insert into product values (2,"Pen",0.67);'
!sqlite3 new.db 'insert into product values (3,"Marker",1.03);'
!sqlite3 new.db 'insert into order_n values (1,2,1,13);'
!sqlite3 new.db 'insert into order_n values (2,3,2,45);'


## Experiments

In in the cell below, experiment with the commands from the slides.

In [ ]:
# !sqlite3 new.db 'select...'

## Bank Data

Now we are going to experiment with more analytical data.

### Download data

```
wget http://www.fdic.gov/bank/individual/failed/banklist.csv
```

### Import into database

#### From command line
```
sqlite3 banks.db
```

#### Inside SQL terminal

```
.mode csv
.import banklist.csv bank
.schema bank
select * from bank limit 10;
```

In [ ]:
!wget http://www.fdic.gov/bank/individual/failed/banklist.csv
!sqlite3 banks.db ".mode csv" ".import banklist.csv bank" 
!sqlite3 banks.db ".schema bank" 
# !sqlite3 banks.db "select ...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

## Db connection in python

Read through the API doc at https://docs.python.org/2/library/sqlite3.html and use that to:

1. List 10 banks
2. List all Chicago Banks
3. List all Wyoming Banks

In [ ]:
# conn = sqlite3.connect...

## List 20 banks

In [ ]:
# results = ...
print(results)

## List Chicago Banks

In [ ]:
# results = ...
print(results)

## List Wyoming Banks

In [ ]:
# results = ...
print(results)

## Use Pandas API to pull in Table data


Read API docs at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html.

Use `pd.read_sql` to pull in Bank data as a DataFrame

In [ ]:
# df = pd.read_sql_query(...
df.head()

## Using Closing Date column to make datetime index

Use https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html and https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DatetimeIndex.html.  You may have to "coerce" any errors.

In [ ]:
# df.index = pd.DatetimeIndex(
#     pd.to_datetime(...
df.head()

## Plot Monthly Bank Failures

See https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Grouper.html for guidance.

In [ ]:
# df.groupby(...

## Joining data

Let's say we want to analyze the bank failure with respect to the state population. For instance, we might want to understand the failure rate per person in a state.  To do this we need to "join" the failure data with a table of state populations.

We can use the data from https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population and https://www.infoplease.com/state-abbreviations-and-state-postal-codes to create a flat table with bank closures and population.





In [ ]:
df_state_codes = pd.read_html(
    'https://www.infoplease.com/state-abbreviations-and-state-postal-codes')[0]
df_state_codes

df_state_pop = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population',
    header=0)[0]
df_state_pop = df_state_pop[['Population estimate, July 1, 2017[4]', 'State or territory']].rename(columns={
    'Population estimate, July 1, 2017[4]': 'population', 'State or territory':'State/District'
})
df_state_pop['population'] = pd.to_numeric(df_state_pop['population'],errors='coerce')
df_state_pop['population'].dtype

In [ ]:
df_pop_codes = df_state_codes.merge(df_state_pop,on='State/District')
df_pop_codes.head()

## Use a similar process to join with bank df

In [ ]:
# df_banks_pop = df_pop_codes.merge(...
# df_banks_pop.index = pd.DatetimeIndex(pd.to_datetime(...
df_banks_pop.head()

## Group by year and state and compute the failures per 1M people per year

In [ ]:
# annual = df_banks_pop.groupby(...).agg({'population':'mean', 'Bank Name':len})

annual['fail_per_cap'] = (annual['Bank Name']/annual['population'])*1000000

annual.reset_index(inplace=True)
annual.head()

# Use seaborn to plot all states by year

See https://seaborn.pydata.org/generated/seaborn.FacetGrid.html

In [ ]:
import seaborn as sns

# g = sns.FacetGrid(...
# g.map(...


## Bonus repeat join using sql

1. Create join tables as csv
2. Load to sqlite
3. Join using sqlite
4. Extract back to Pandas